In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data/infectious-diseases-by-county-year-and-sex.csv")

In [ ]:
df.head()

In [ ]:
diseases = df['Disease'].unique()

d_list = []
for d in diseases:
    tmp_df = df[df['Disease'] == d]
    total_tmp_df = tmp_df[tmp_df['Sex'] == 'TOTAL']
    d_list.append(total_tmp_df)

    total_sum = []
    for c in total_tmp_df['Year'].unique():
        #print(c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum())
        total_sum.append((c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum()))
    total_sum = pd.DataFrame(total_sum, columns=['Year', 'Cases'])
    
    fig, ax = plt.subplots(figsize=(25,14))
    total_sum.plot(x='Year', y='Cases', ax=ax)
    plt.title(d)
    plt.savefig('images/'+ d.replace('.', '_'))
    plt.close()

In [ ]:
d_list[0]

In [ ]:
county_info = pd.read_csv("data/county_info.csv", sep=';')

county_info['Per capita income'] = county_info['Per capita income'].apply(lambda x: x.replace('$', ''))
county_info['Median household income'] = county_info['Median household income'].apply(lambda x: x.replace('$', ''))
county_info['Median family income'] = county_info['Median family income'].apply(lambda x: x.replace('$', ''))

county_info['Per capita income'] = county_info['Per capita income'].astype(int)
county_info['Median household income'] = county_info['Median household income'].astype(int)
county_info['Median family income'] = county_info['Median family income'].astype(int)

county_info.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
county_info.plot(kind='bar', x='County', y='Per capita income', ax=ax)
plt.show()
county_info.plot(kind='bar', x='County', y='Median household income', ax=ax)
county_info.plot(kind='bar', x='County', y='Median family income', ax=ax)


In [ ]:
## dikke rik